# Lagrangian Paper Tutorial
This notebook provides a step-by-step demonstration/tutorial based on the Lagrangian paper.

## Introduction
A short flash-talk style introduction to the Lagrangian paper to ensure we are on the same page regarding the example.

## Models
- Overview of HuggingFace library.
- How to find off-the-shelf transformer models (e.g., BART-L).
- Example usage of a HuggingFace model.

In [ ]:
# Import HuggingFace libraries
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load a pre-trained model and tokenizer (e.g., BART-Large)
model_name = 'facebook/bart-large'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example usage
text = "This is a sample input."
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

model_name = "JoseEliel/BART-Lagrangian"
model = BartForConditionalGeneration.from_pretrained(model_name)
hf_tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)


## Dataset
- Discussion on data generation considerations:
  - Data distribution.
  - Tokenization choices.
- Example of tokenizing a dataset.

In [ ]:
# Example: Tokenizing a dataset
dataset = ["Example sentence 1.", "Example sentence 2."]
tokenized_dataset = [tokenizer(sentence, return_tensors="pt") for sentence in dataset]
print(tokenized_dataset)

## Training
- Mention available resources: SUPR/NAISS -> Alvis.
- Example of training a model.

In [ ]:
# Example: Training a model (pseudo-code)
# Define training loop and optimizer
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)
for epoch in range(3):
    for batch in tokenized_dataset:
        outputs = model(**batch, labels=batch['input_ids'])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

## Evaluation
- Generating output from the model.
- Discussion on evaluation choices:
  - Existing or novel metrics.
  - Embedding analysis.
  - Out-of-distribution tests.

In [ ]:
# Example: Generating output
test_text = "This is a test input."
test_inputs = tokenizer(test_text, return_tensors="pt")
test_outputs = model.generate(**test_inputs)
print(tokenizer.decode(test_outputs[0], skip_special_tokens=True))